Implements algorithm 3 (ModMod) from Algorithms for "Approx Min of the Difference Between Submodular Fncs with Applications."

For prob_ssm

In [1]:
import torch
from torch import distributions as dist

import itertools
import pickle
import operator
import importlib

import random
import math
import numpy as np
import pandas as pd
import scipy

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('white')
sns.set_context('paper')
# Plot adjustments:
plt.rcParams.update({'ytick.labelsize': 15})
plt.rcParams.update({'xtick.labelsize': 15})
plt.rcParams.update({'axes.labelsize': 35})
plt.rcParams.update({'legend.fontsize': 30})
plt.rcParams.update({'axes.titlesize': 16})

from gptorch import kernels, models

import helpers, opt

In [2]:
with open('../inputs/phoq.pkl', 'rb') as f:
    t = pickle.load(f)

X = t[0] # one-hot encoding of X
T = t[1] # tokenized encoding of X
y = t[2].values

In [3]:
wt = helpers.decode_X(X[150614])  # wt as string
aas = 'ARNDCQEGHILKMFPSTWYV'

In [4]:
seq_to_x = {}
for i, x in enumerate(X):
    seq = helpers.decode_X(x)
    seq_to_x[seq] = i

In [5]:
singles = [wt[:i] + aas[j] + wt[i+1:] for i in range(4) for j in range(20)]
len(singles)

80

In [6]:
singles = [wt[:i] + aas[j] + wt[i+1:] for i in range(4) for j in range(20)]
train_seqs = [wt] + singles
train_inds = [seq_to_x[seq] for seq in train_seqs]
train_inds += list(np.random.choice(len(X), 100, replace=True)) # generate random indices for 100 X's to sample from

X_train = X[train_inds]
y_train = y[train_inds]
X_test = X
y_true = y

In [7]:
ground = [(aa, i) for aa in aas for i in range(4)]

In [8]:
def seqs_from_set(chosen, L):
    pos = [[c[0] for c in chosen if c[1] == p] for p in range(L)]
    return [''.join(s) for s in itertools.product(*pos)]

X_sampled = [X_train]
y_sampled = [y_train]

In [9]:
helpers = importlib.reload(helpers)
opt = importlib.reload(opt)

n_start = 35
max_its = 30
rounds = 4
L = 4
n = 100


observed_inds = [train_inds]
X_sampled = [X[train_inds]]
y_sampled = [y[train_inds]]


libraries = []
histories = []

for rou in range(rounds):
    print('Round %d' %rou)
    inds = sorted(set(itertools.chain(*observed_inds)))
    print("here0")
    dic, _ = helpers.get_predictions(X[inds], y[inds], X_test, its=500)    
    print()
    print("here0.5")
    best_loss = 0.0
    best_X = None
    best_h = None
    for i in range(max_its):
        np.random.seed(i)
        center = np.random.choice(len(ground), n_start, replace=False)
        center = [ground[c] for c in center]
        print("here1")
        chosen, h = opt.mod_mod(ground, opt.obj_LHS, opt.obj_RHS_edit, center, 
                           fn_args=(L, dic), g_args=(L, dic, n), verbose=False)
        if h[-1] < best_loss:
            best_loss = h[-1]
            best_X = chosen
            best_h = h
    print("here2")
    libraries.append(best_X)
    histories.append(best_h)
    print("here3")
    seqs = seqs_from_set(best_X, L)
    inds = np.random.choice(len(seqs), n, replace=True)
    sampled_seqs = [seqs[i] for i in inds]
    inds = [seq_to_x[s] for s in sampled_seqs]
    print("here4")
    X_sampled.append(X[inds])
    y_sampled.append(y[inds])
    observed_inds.append(inds)
    print("here5")

Round 0
here0
Iteration 500 of 500	NLML: 43.0758	sn: 0.150327	
here0.5
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here2
here3
here4
here5
Round 1
here0
Iteration 500 of 500	NLML: 61.5336	sn: 0.142351	
here0.5
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here2
here3
here4
here5
Round 2
here0
Iteration 500 of 500	NLML: 63.1081	sn: 0.140175	
here0.5
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here2
here3
here4
here5
Round 3
here0
Iteration 500 of 500	NLML: 63.1081	sn: 0.140175	
here0.5
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
here1
he

In [10]:
libraries

[[('I', 0),
  ('S', 0),
  ('D', 0),
  ('L', 0),
  ('Q', 0),
  ('H', 0),
  ('G', 0),
  ('C', 1),
  ('L', 1),
  ('I', 1),
  ('D', 1),
  ('M', 1),
  ('V', 1),
  ('W', 1),
  ('Q', 1),
  ('H', 1),
  ('K', 1),
  ('S', 1),
  ('S', 2),
  ('A', 2),
  ('Y', 2),
  ('G', 2),
  ('C', 2),
  ('Q', 2),
  ('E', 2),
  ('F', 2),
  ('N', 2),
  ('T', 2),
  ('E', 3),
  ('D', 3),
  ('S', 3),
  ('L', 3),
  ('V', 3),
  ('M', 3),
  ('W', 3)],
 [('I', 0),
  ('L', 0),
  ('P', 0),
  ('K', 0),
  ('S', 0),
  ('W', 1),
  ('F', 1),
  ('Y', 1),
  ('H', 1),
  ('K', 1),
  ('S', 1),
  ('V', 1),
  ('T', 1),
  ('K', 2),
  ('Y', 2),
  ('H', 2),
  ('S', 2),
  ('P', 2),
  ('N', 2),
  ('I', 2),
  ('A', 2),
  ('T', 2),
  ('E', 2),
  ('V', 2),
  ('L', 2),
  ('S', 3),
  ('E', 3),
  ('G', 3),
  ('K', 3),
  ('I', 3),
  ('H', 3),
  ('D', 3),
  ('C', 3),
  ('T', 3),
  ('V', 3)],
 [('I', 0),
  ('L', 0),
  ('P', 0),
  ('K', 0),
  ('S', 0),
  ('W', 1),
  ('F', 1),
  ('Y', 1),
  ('H', 1),
  ('K', 1),
  ('S', 1),
  ('V', 1),
  ('T', 1),
  

In [11]:
histories

[[tensor(-15.9069), tensor(-15.9069)],
 [tensor(-42.4598), tensor(-42.4598)],
 [tensor(-40.9212), tensor(-40.9212)],
 [tensor(-40.9212), tensor(-40.9212)]]

In [ ]:
df = pd.DataFrame(np.concatenate(y_sampled), columns=['y'])
df['round'] = [0] * (81 + n) + [1] * n + [2] * n + [3] * n + [4] * n
df.head()


In [ ]:
_ = sns.swarmplot(data=df, x='round', y='y')

In [ ]:
[max(yy) for yy in y_sampled]

In [ ]:
[opt.get_N(lib, L) for lib in libraries]